In [ ]:
import sys
sys.path.insert(1,'../../dependencies')
from pathlib import Path
import itertools

import numpy as np
import geopandas as gpd
import pandas as pd
import datetime as dt

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib.transforms import offset_copy
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter

import flopy
import rasterio
from rasterio.io import MemoryFile
from shapely.geometry import Polygon

from dataretrieval import nwis

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy_addons.scalebar import draw_scale
from cartopy_addons.annotations import label_line

#  Suppress warnings issued by Cartopy that come up WRT Shapely 2.0
import warnings
warnings.filterwarnings('ignore')


The first half of this notebook is based on the [Cartopy tutorial created by Project Pythia](https://foundations.projectpythia.org/core/cartopy.html), which introduces the core concepts behind Cartopy. 

The second half shows how to make maps and figures that comply with the Standards Guide for USGS illustrations.

# Cartopy

From the Cartopy website:

>Cartopy is a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses.
>
>Cartopy makes use of the powerful PROJ.4, NumPy and Shapely libraries and includes a programmatic interface built on top of Matplotlib for the creation of publication quality maps.
>
>Key features of Cartopy are its object-oriented projection definitions, and its ability to transform points, lines, vectors, polygons and images between those projections.


Cartopy allows us to make figures that are (theoretically) publication-ready, without having to touch any other software!

## map projections and `GeoAxes`

## Extending matplotlib `axes` into georeferenced `GeoAxes`

A figure in Matplotlib has two elements: a `Figure` object, and a list of one or more `Axes` objects (subplots).

Since we imported `cartopy.crs`, we now have access to Cartopy’s Coordinate Reference System, which contains many geographical projections. We can specify one of these projections for an Axes object to convert it into a GeoAxes object. This will georeference the subplot.

In [ ]:
fig = plt.figure(figsize=(11, 8.5))
ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree(central_longitude=-75))
ax.set_title("A Geo-referenced subplot, Plate Carree projection");

Although the figure seems empty, but it has been georeferenced using a map projection provided by Cartopy’s `crs` (coordinate reference system) class (which we are using with the alias `ccrs`). 

We can now add in cartographic features, in the form of shapefiles, to our subplot. One such cartographic feature is coastlines, which can be added to our subplot using the callable GeoAxes method simply called coastlines.

In [ ]:
ax.coastlines()
fig # calling the figure again

## adding features from Cartopy's library

Cartopy provides other cartographic features via its features class, which we imported with the alias `cfeature`. These cartographic features are laid out as data in shapefiles. The shapefiles are downloaded when their cartographic features are used for the first time in a script or notebook, and they are downloaded from https://www.naturalearthdata.com/. Once downloaded, they “live” in your ~/.local/share/cartopy directory (note the ~ represents your home directory, likely your User directory).

We can add these features to our subplot via the `add_feature` method for the `GeoAxes`. We can define attributes using arguments, similar to Matplotlib’s `plot` method. 

A list of the various Natural Earth shapefiles can be found at https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html. In this example, we add borders and U. S. state lines to our subplot.

**NOTE**: I'm not 100% sure about making use of Cartopy's features when making report-ready USGS figures. But they are definitely great for quickly putting togther a figure!

In [ ]:
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
ax.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='brown')
fig

We can also use the `stock_img` method to add a pre-created background to a plot.

In [ ]:
ax.stock_img()
fig

## Creating regional maps

Now, we'll use Cartopy’s `set_extent` method to restrict the map coverage to a North American view. We'll also set a lower resolution for coastlines. In addition, we'll plot the latitude and longitude lines.

Natural Earth defines three resolutions for cartographic features, specified as the strings “10m”, “50m”, and “110m”. Only one resolution can be used at a time, and the higher the number, the less detailed the feature becomes. You can view the documentation for this functionality here: https://www.naturalearthdata.com/downloads/.

In [ ]:
projPC = ccrs.PlateCarree()
lonW = -140
lonE = -40
latS = 15
latN = 65
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
res = '50m'

In [ ]:
fig = plt.figure(figsize=(11, 8.5))
ax = plt.subplot(1, 1, 1, projection=projPC)
ax.set_title('Plate Carree')
gl = ax.gridlines(
    draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--'
)
ax.set_extent([lonW, lonE, latS, latN], crs=projPC)
ax.coastlines(resolution=res, color='black')
ax.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='brown')
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='blue');

We may want to use a different projection, to avoid the skew caused by PlateCarree.

In [ ]:
projStr = ccrs.Stereographic(central_longitude=cLon, central_latitude=cLat)
fig = plt.figure(figsize=(11, 8.5))
ax = plt.subplot(1, 1, 1, projection=projStr)
ax.set_title('Stereographic')
gl = ax.gridlines(
    draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--'
)
ax.set_extent([lonW, lonE, latS, latN], crs=projPC)
ax.coastlines(resolution=res, color='black')
ax.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='brown')
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='blue');

**NOTE**: even though the calls to the subplot method can use different projections, it's good to set calls to `set_extent` with PlateCarree. This ensures that the values we passed into `set_extent` will be transformed from degrees into the values appropriate for the projection we use for the map.


Also, Lat/lon labeling for projections other than Mercator and PlateCarree isn't always great. Work still needs to be done to improve the placement of labels, but we'll be using some special methods when creating our USGS maps later.

## A regional map of New York

In [ ]:
latN = 45.2
latS = 40.2
lonW = -80.0
lonE = -71.5
cLat = (latN + latS) / 2
cLon = (lonW + lonE) / 2
projLccNY = ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)

### Add some predefined features

Some cartographical features are predefined as constants in the cartopy.feature package. The resolution of these features depends on the amount of geographical area in your map, specified by set_extent.

**Be patient!** When plotting a small geographical area with Cartopy features, the high-resolution “10m” shapefiles are used by default. As a result, these plots take longer to create, especially if the shapefiles are not yet downloaded from Natural Earth. Similar issues can occur whenever a GeoAxes object is transformed from one coordinate system to another.

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = plt.subplot(1, 1, 1, projection=projLccNY)
ax.set_extent([lonW, lonE, latS, latN], crs=projPC)
ax.set_facecolor(cfeature.COLORS['water'])
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='--')
ax.add_feature(cfeature.LAKES, alpha=0.5)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.RIVERS)
ax.set_title('New York and Vicinity');

**NOTE**: For high-resolution Natural Earth shapefiles such as this, while we could add Cartopy’s `OCEAN` feature, it currently takes much longer to render on the plot. (You can create your own version of this example, with the `OCEAN` feature added, to see for yourself how much more rendering time is added.)

Instead, we first set the facecolor of the entire subplot to match that of water bodies in Cartopy. When we layer on the `LAND` feature, pixels that are not part of the LAND shapefile remain in the water facecolor, which is the same color as the OCEAN.

## Plotting data on maps

Creating some data that spans the whole globe.

In [ ]:
lon, lat = np.mgrid[-180:181, -90:91]
data = 2 * np.sin(3 * np.deg2rad(lon)) + 3 * np.cos(4 * np.deg2rad(lat))
plt.contourf(lon, lat, data)
plt.colorbar();

Plotting data on a Cartesian grid is equivalent to plotting data in the PlateCarree projection, where meridians and parallels are all straight lines with constant spacing. As a result of this simplicity, the global datasets we use often begin in the PlateCarree projection.

We can plot these data values as a contour on a `GeoAxes` map. To do so, we must specify the `transform` keyword argument, which specifies the projection type used by our data. The `transform` keyword can be given to all matplotlib plotting methods. The projection type specified in `contourf` will be transformed into the projection type specified in the `subplot` method.

In [ ]:
projMoll = ccrs.Mollweide(central_longitude=0)
fig = plt.figure(figsize=(11, 8.5))
ax = plt.subplot(1, 1, 1, projection=projMoll)
ax.coastlines()
dataplot = ax.contourf(lon, lat, data, transform=ccrs.PlateCarree())
plt.colorbar(dataplot, orientation='horizontal');

# USGS Figures

Okay, now that we know how Cartopy works, we can use it with Geopandas and Rasterio to quickly create USGS-style figures. And most importantly, if the project data ever changes, we can very quickly update our figures.

## Univers fonts (do you have them?)

In order to make USGS figures, you will need the Univers fonts installed. Let's check for them!

In [ ]:
import matplotlib.font_manager as fm 
# grab the file names of system fonts that have the word 'univers' in them
univers_fonts = [f for f in fm.findSystemFonts(fontpaths=None, fontext='ttf') if 'univers' in f.lower()]

# now get Matplotlib's names for those fonts, these are what you use when assigning fonts 
univers_names = [fm.FontProperties(fname=f).get_name() for f in univers_fonts]

dict(zip(univers_names,univers_fonts))

If they are not currently in your system fonts, you can download them from @theCore by searching for "Typography". To install them on Windows, follow [these instructions](https://support.microsoft.com/en-gb/office/add-a-font-b7c5f17c-4426-4b53-967f-455339c564c1).


## Making an example map of Long Island

### SPN map specs

These are a set of SPN specifications originally put together by Joe Hughes, and can be found here: https://github.com/jdhughes-usgs/usgs_figure_specifications. 

I have modified them slightly, and you can make changes yourself according to any SPN spec updates that might happen in the future.

In [ ]:
def set_map_specifications():
    rc_dict = {'font.family': 'Univers 47 Condensed Light',
               'font.size': 7,
               'axes.labelsize': 9,
               'axes.titlesize': 9,
               'axes.linewidth': 0.5,
               'xtick.labelsize': 7,
               'xtick.top': True,
               'xtick.bottom': True,
               'xtick.major.size': 7.2,
               'xtick.minor.size': 3.6,
               'xtick.major.width': 0.5,
               'xtick.minor.width': 0.5,
               'xtick.direction': 'in',
               'ytick.labelsize': 7,
               'ytick.left': True,
               'ytick.right': True,
               'ytick.major.size': 7.2,
               'ytick.minor.size': 3.6,
               'ytick.major.width': 0.5,
               'ytick.minor.width': 0.5,
               'ytick.direction': 'in',
               'pdf.fonttype': 42,
               'savefig.dpi': 300,
               'savefig.transparent': True,
               'legend.fontsize': 9,
               'legend.frameon': True,
               'legend.fancybox': False,
               'legend.fontsize': 7.0,
               'legend.framealpha': 1.,
               'legend.markerscale': 1.
               }
    mpl.rcParams.update(rc_dict)
set_map_specifications()


### First, let's make a basemap that we'll overlay data on top of

In [ ]:
# decimal degree extents for our map
xmin = -74.2
xmax = -71.7
ymin = 40.5
ymax = 41.3

Let's load in some shapefiles, clipping some of the really big ones to our extent. That minimizes the time it takes to render the figure.

We're using [EPSG 4456](https://epsg.io/4456), which is for the New York State Plane in ft for Long Island.

In [ ]:
# this function clips a shapefie to a bounding box based on the map extent we set below

def clip_to_li_area(shp, xmin, xmax, ymin, ymax, epsg=4456):
    # create a Shapely polygon of the bounding box
    polygon = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)])
    # create a geodataframe from the bounding box polygon
    poly_gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs=ccrs.PlateCarree()).to_crs(shp.crs)
    # clip and output with epsg specified above
    shp_clip = shp.clip(poly_gdf.geometry).to_crs(epsg=epsg)
    return shp_clip

In [ ]:
counties = gpd.read_file('./data/mapping/shapefiles/Counties_Shoreline.shp').to_crs(epsg=4456)

coastal_states = gpd.read_file('./data/mapping/shapefiles/coastal_states.shp')
coastal_states_clip = clip_to_li_area(coastal_states, xmin, xmax, ymin, ymax)

state_lines = gpd.read_file('./data/mapping/shapefiles/NorthEastStateInset_NoCoasts.shp')
state_lines_clip = clip_to_li_area(state_lines, xmin, xmax, ymin, ymax)

# the Hudson, East, and Bronx R have their own detailed shapefiles
hudson_river = gpd.read_file('./data/mapping/shapefiles/HudsonRiver_EastRiver_BronxRiver.shp')

# load in some water bodies, concatenating them together as we load
water_bodies = pd.concat([gpd.read_file('./data/mapping/shapefiles/NHD_Waterbody_NYNJ.shp'),
                          gpd.read_file('./data/mapping/shapefiles/NHD_Waterbody_CTRI.shp')])
water_bodies_clip = clip_to_li_area(water_bodies, xmin, xmax, ymin, ymax)

In [ ]:
# some of our our shapefiles are in PlateCarree, while others are in EPSG:4456
basemap_crs = ccrs.PlateCarree()
element_crs = ccrs.epsg(4456)

Let's first plot the states and LI counties with slightly different colors using the Geopandas `plot` method.

Notice that we're using the `transform` kwarg to tell Cartopy what CRS the shapefile is in.

We're also using `rasterized=True` so that if/when we plot this out to PDF, the shapefiles will be in rasters rather than vectors. When done for shapefiles that have many features, this VASTLY improves the rendering of figures in a final report PDF, eliminating the loading times you may have experienced while scrolling through a PDF if highly vectorized figures.

In [ ]:
fig = plt.figure(dpi=150,figsize=(7.5,10))
ax = fig.add_subplot(1,1,1,projection=ccrs.Mercator())

ax.set_extent([xmin, xmax, ymin, ymax], crs=basemap_crs)
ax.fill_between([xmin,xmax],[ymax,ymax],[ymin,ymin],color='#C2E8FA',transform=basemap_crs,zorder=-1)

# plot non-LI NY counties
other_cos = ['New York','Bronx','Westchester','Rockland','Orange','Richmond','Putnam']
counties[counties['NAME'].isin(other_cos)].plot(
    ax=ax,fc='#B4B3B4',ec='grey',lw=0,transform=element_crs,zorder=0,rasterized=True, legend=False)

# plot other states
coastal_states_clip[coastal_states_clip['STATE_COD'].isin(['NJ','CT','RI'])].plot(
    ax=ax,fc='#B4B3B4',ec='none',lw=1,transform=element_crs,zorder=0,rasterized=True, legend=False)

# plot LI counties
counties[counties['NAME'].isin(['Kings','Queens','Nassau','Suffolk'])].plot(
    ax=ax,fc='whitesmoke',ec='grey',lw=0,transform=element_crs,zorder=0,rasterized=True, legend=False)
state_lines_clip.plot(
    ax=ax, fc='none', ec='k', ls='dashdot', lw=1, transform=element_crs, zorder=0, rasterized=True, legend=False)
hudson_river.plot(
    ax=ax, fc='#C2E8FA', ec='none', transform=element_crs, zorder=0, rasterized=True, legend=False)
water_bodies_clip.plot(
    ax=ax, fc='#C2E8FA', ec='none', transform=element_crs, zorder=0, rasterized=True, legend=False)


That looks pretty nice, but we don't have much context, like lat/lon ticks, labels, scale bar, and a note on credits. Let's add those.

For the ticks, we'll be using the `LongitudeFormatter` and `LatitudeFormatter` available for Cartopy, which are a large imrovement over the default tickers we saw above.

In [ ]:
# TICKS
ax.set_xticks(np.arange(-74.0, -71.5, .5), crs=ccrs.PlateCarree())
ax.set_yticks(np.arange(40.50, 41.5, 0.5), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True,dms=True)
lat_formatter = LatitudeFormatter(dms=True)
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
ax.tick_params(axis='both',which='both',labelbottom=False,labeltop=True,zorder=3)

fig

In [ ]:
# State labels
ax.annotate('NEW\nJERSEY', xy=(.05,.5), xycoords='axes fraction',
            ha='center',va='bottom', family='Times New Roman', weight='bold',fontsize=8)
ax.annotate('NEW\nYORK', xy=(.18,.8), xycoords='axes fraction',
            ha='center',va='bottom', family='Times New Roman', weight='bold',fontsize=8)
ax.annotate('CONNECTICUT', xy=(.38,.9), xycoords='axes fraction',
            ha='center',va='bottom', family='Times New Roman', weight='bold',fontsize=8)

# # water bodies
ax.annotate('LONG ISLAND SOUND', color='#0668B3', xy=(.5,.7), xycoords='axes fraction',
            ha='center',va='bottom',rotation=18,fontstyle='italic',family='Times New Roman',fontsize=8)
ax.annotate('ATLANTIC OCEAN', color='#0668B3', xy=(.5,.1), xycoords='axes fraction',
            ha='center',va='bottom',rotation=18,fontstyle='italic',family='Times New Roman',fontsize=8)

fig

Scalebars can be a real pain to create, but I've updated the scalebar function created by the [Shakemap](https://github.com/usgs/shakemap) folks to make a scalebar that includes both miles and kilometers. 

You might have to play around with the `end_off` kwarg to define how much offset you need on the last number of the scalebar (I haven't figured out how to avoid the manual tinkering, I welcome any suggestions you might have!). 

The python script that we're using for this is located in `dependencies/cartopy_addons`.

In [ ]:
# SCALEBAR
# first define how long a distance you would like to represent
length = 15.
# then define how many divisions
divisions = 3
# draw the miles
draw_scale(ax, pos='lr', locx=0.75, locy=-0.1, units='mi',lw=.5,font_size=7,
        direction='up',divisions=divisions,bar_length=length,end_off=1.5)
# then draw the kilometers
draw_scale(ax, pos='lr', locx=0.75, locy=-0.1, units='km',lw=.5,font_size=7,
        direction='down',divisions=divisions,bar_length=length,end_off=1.5)

fig

Finally, let's add the credits for the basemap parts.

In [ ]:
# Credit note
ax.annotate(
    'Base map modified from U.S. Geological Survey National Atlas digital data files \
    \nNew York [Long Island] State Plane projection \
    \nNorth American Datum of 1988',
    xy=(0,-0.01),
    xycoords='axes fraction',
    ha='left',
    va='top',
)
fig

### combine it all into a single basemap function!

In [ ]:
basemap_crs = ccrs.PlateCarree()
element_crs = ccrs.epsg(4456)

def make_base(ax,plot_states=True,basemap_crs=basemap_crs,element_crs=element_crs,xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax):
    ax.set_extent([xmin, xmax, ymin, ymax], crs=basemap_crs)
    ax.fill_between([xmin,xmax],[ymax,ymax],[ymin,ymin],color='#C2E8FA',transform=basemap_crs,zorder=-1)
    
    # plot non-LI counties
    if plot_states:
        other_cos = ['New York','Bronx','Westchester','Rockland','Orange','Richmond','Putnam']
        counties[counties['NAME'].isin(other_cos)].plot(
            ax=ax,fc='#B4B3B4',ec='grey',lw=0,transform=element_crs,zorder=0,rasterized=True, legend=False)
        # plot other states
        coastal_states_clip[coastal_states_clip['STATE_COD'].isin(['NJ','CT','RI'])].plot(
            ax=ax,fc='#B4B3B4',ec='none',lw=1,transform=element_crs,zorder=0,rasterized=True, legend=False)
        # plot LI counties
        counties[counties['NAME'].isin(['Kings','Queens','Nassau','Suffolk'])].plot(
            ax=ax,fc='whitesmoke',ec='grey',lw=0,transform=element_crs,zorder=0,rasterized=True, legend=False)
        state_lines_clip.plot(
            ax=ax, fc='none', ec='k', ls='dashdot', lw=1, transform=element_crs, zorder=0, rasterized=True, legend=False)
        hudson_river.plot(
            ax=ax, fc='#C2E8FA', ec='none', transform=element_crs, zorder=0, rasterized=True, legend=False)
        water_bodies_clip.plot(
            ax=ax, fc='#C2E8FA', ec='none', transform=element_crs, zorder=0, rasterized=True, legend=False)
    
    # TICKS
    ax.set_xticks(np.arange(-74.0, -71.5, .5), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(40.50, 41.5, 0.5), crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True,dms=True)
    lat_formatter = LatitudeFormatter(dms=True)
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.tick_params(axis='both',which='both',labelbottom=False,labeltop=True,zorder=3)
    
    # State labels
    ax.annotate('NEW\nJERSEY', xy=(.05,.5), xycoords='axes fraction',
                ha='center',va='bottom', family='Times New Roman', weight='bold',fontsize=8)
    ax.annotate('NEW\nYORK', xy=(.15,.8), xycoords='axes fraction',
                ha='center',va='bottom', family='Times New Roman', weight='bold',fontsize=8)
    ax.annotate('CONNECTICUT', xy=(.4,.9), xycoords='axes fraction',
                ha='center',va='bottom', family='Times New Roman', weight='bold',fontsize=8)
    
    # # water bodies
    ax.annotate('LONG ISLAND SOUND', color='#0668B3', xy=(.5,.7), xycoords='axes fraction',
                ha='center',va='bottom',rotation=18,fontstyle='italic',family='Times New Roman',fontsize=8)
    ax.annotate('ATLANTIC OCEAN', color='#0668B3', xy=(.5,.1), xycoords='axes fraction',
                ha='center',va='bottom',rotation=18,fontstyle='italic',family='Times New Roman',fontsize=8)
    
    # SCALEBAR
    length = 15.
    divisions = 3
    draw_scale(ax, pos='lr', locx=0.75, locy=-0.1, units='mi',lw=.5,font_size=7,
            direction='up',divisions=divisions,bar_length=length,end_off=1.5)
    draw_scale(ax, pos='lr', locx=0.75, locy=-0.1, units='km',lw=.5,font_size=7,
            direction='down',divisions=divisions,bar_length=length,end_off=1.5)
    
    # Credit note
    ax.annotate(
        'Base map modified from U.S. Geological Survey National Atlas digital data files \
        \nNew York [Long Island] State Plane projection \
        \nNorth American Datum of 1988',
        xy=(0,-0.01),
        xycoords='axes fraction',
        ha='left',
        va='top',
    )

## Adding some well locations from NWIS to the basemap

Let's grab all the recent NWIS wells in the Long Island counties. `dataretrieval` allows you to use 5-digit county codes, with the [FIPS state code](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) as the first two digits. The remaining three digits can be found here: https://help.waterdata.usgs.gov/code/county_query?fmt=html.



In [ ]:
county_codes = [f'36{cd}' for cd in ['047', '081', '059', '103']]
# get info about all sites with data in 2023
info, meta = nwis.get_info(
    countyCd=county_codes,
    siteType='GW',
    startDt='2023-01-01'
)

# add them to a geodataframe
wells = gpd.GeoDataFrame(
    info[['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va']], 
    geometry=gpd.points_from_xy(info['dec_long_va'], info['dec_lat_va']), 
    crs=ccrs.PlateCarree()
)

In [ ]:
fig = plt.figure(dpi=150,figsize=(7.5,10))

ax = fig.add_subplot(1,1,1,projection=ccrs.Mercator())

make_base(ax, plot_states=True)

wells.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', lw=.5, facecolor='tab:blue', markersize=5, alpha=.8)

## Now let's add a raster of 2019 population estimates

We'll load this population density raster (population per square mile) using rasterio and plot it with `pcolormesh`. 

In [ ]:
with rasterio.open('./data/mapping/rasters/li_pop2019.tif', 'r') as src:
    raster_epsg = src.crs.to_epsg()
    pop2019 = src.read(1)
    # save the dimensions of the raster
    height = pop2019.shape[0]
    width = pop2019.shape[1]
    # create meshgrids with those dimensions
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    # transform those meshgrids to Xs and Ys using the geospatial transform stored in the raster
    xs, ys = rasterio.transform.xy(src.transform, rows, cols)

In [ ]:

fig = plt.figure(figsize=(7.5,10), dpi=150)
ax = fig.add_subplot(1,1,1,projection=ccrs.Mercator())

make_base(ax, plot_states=True)

# plot the raster array
mesh = ax.pcolormesh(
    xs, ys,
    pop2019,
    cmap='Reds',
    vmin=0,vmax=50000,
    transform=ccrs.epsg(raster_epsg),
    rasterized=True,
)

Hm, we don't want the raster blocking all the rest of our map elements though! We'll use rasterio to create a raster "in memory", masking out the parts we don't want.

In [ ]:
# first we need to grab the metadata from the raster
with rasterio.open('./data/mapping/rasters/li_pop2019.tif', 'r') as src:
    ras_meta = src.profile
    
# our mask will be created using the Long Island counties shapefile
mask_shp = counties.loc[counties['NAME'].isin(['Kings','Queens','Nassau','Suffolk'])]

# now let's create that "in memory" raster
with MemoryFile() as memfile:
    
    # write the population data to the raster
    with memfile.open(**ras_meta) as dst:
        dst.write(pop2019, 1)
        
        # mask the raster using the shapefile
        pop2019_mask, _ = rasterio.mask.mask(
            dataset=dst,
            shapes=mask_shp.loc[:,'geometry'],
            all_touched=True,
            nodata=np.nan,
            indexes=1,
            
        ) 

Now let's plot it again, and add the NWIS wells this time.

In [ ]:
fig = plt.figure(figsize=(7.5,10), dpi=300)
ax = fig.add_subplot(1,1,1,projection=ccrs.Mercator())

make_base(ax, plot_states=True)

# now plot the 
mesh = ax.pcolormesh(
    xs, ys,
    pop2019_mask,
    cmap='Reds',
    vmin=vmin,vmax=60000,
    transform=ccrs.epsg(raster_epsg),
    rasterized=True,
)

# and add the wells this time
wells.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', lw=.5, facecolor='tab:blue', markersize=5, alpha=.8)

Now all we are missing is an Explanation. This can be VERY finicky in python unfortunately, but once you have it figured out for one figure, it isn't that hard to adjust for future ones.

Creating a nice title is especially difficult, and we will need to create a `class` to do so.

In [ ]:
class LegendTitle(object):
    def __init__(self, text_props=None):
        self.text_props = text_props or {}
        super(LegendTitle, self).__init__()

    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        x0, y0 = handlebox.xdescent, handlebox.ydescent
        title = mpl.text.Text(x0, y0, orig_handle, **self.text_props)
        handlebox.add_artist(title)
        return title

In [ ]:
fig = plt.figure(figsize=(7.5,10), dpi=300)
ax = fig.add_subplot(1,1,1,projection=ccrs.Mercator())
make_base(ax, plot_states=True)
mesh = ax.pcolormesh(
    xs, ys,
    pop2019_mask,
    cmap='Reds',
    vmin=vmin,vmax=60000,
    transform=ccrs.epsg(raster_epsg),
    rasterized=True,
)
wells.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', lw=.5, facecolor='tab:blue', markersize=5, alpha=.8)



# add the explanation
# first set how large your colorbar will be
cbx = 0.88
cby = .05
cbw = .05
cbh = .28
# then set how many lines your legend needs and extra side padding. this will be trial and error! 
leg_rows = 6
leg_width_extra = 12
handles = [Line2D([], [], linewidth=0, markeredgecolor='k', markeredgewidth=.5, markerfacecolor='tab:blue', marker='o', markersize=3)] + \
          ['Pop. density,'] + ['per sq mile'] + ['']*leg_rows
labels = ['NWIS well'] + [' '*(len(handles[1])+leg_width_extra)]*(leg_rows+1)
leg = ax.legend(
    handles,labels,loc='lower left',
    fontsize=7,
    bbox_to_anchor=(cbx-.05,cby-.04),
    title='EXPLANATION',
    edgecolor='k',
    handleheight=1,
    title_fontproperties={'family': 'Univers 67 Condensed',
                          'size': 8},
    prop={'family':'Univers 57 Condensed'},
    handler_map={str: LegendTitle({'fontsize': 7, 'family':'Univers 67 Condensed'})})
leg.get_frame().set_linewidth(.5)
cax = ax.inset_axes((cbx, cby, cbw, cbh),zorder=99)
fig.colorbar(mesh, cax=cax, orientation='vertical')

## This figure can be saved as a PDF and easily opened in Adobe Illustrator for any final touches you might need.

In [ ]:
fig.savefig('./data/mapping/test_figure2.pdf', bbox_inches='tight', dpi=300)

What happens to the PDF if you comment out the `rasterized` above in `pcolormesh`?

## BONUS: Maybe we want to adjust the lines to be a very specific pattern specified by SPN?

The formatting of more complicated line styles is outlined here: https://matplotlib.org/stable/gallery/lines_bars_and_markers/linestyles.html. 

In a plot method, the keyword arg `linestyle` (or alias `ls`) is fed a pattern of line and space lengths.

For example, `(0, (3, 10, 1, 15))` means (3pt line, 10pt space, 1pt line, 15pt space) with no offset, while `(5, (10, 3))`, means (10pt line, 3pt space), but skip the first 5pt line.

In [ ]:
fig = plt.figure(dpi=150,figsize=(7.5,10))
ax = fig.add_subplot(1,1,1,projection=ccrs.Mercator())

ax.set_extent([xmin, xmax, ymin, ymax], crs=basemap_crs)
ax.set_facecolor('#C2E8FA')

# plot non-LI NY counties
other_cos = ['New York','Bronx','Westchester','Rockland','Orange','Richmond','Putnam']
counties[counties['NAME'].isin(other_cos)].plot(
    ax=ax,fc='#B4B3B4',ec='grey',lw=0,transform=element_crs,zorder=0,rasterized=True, legend=False)

# plot other states
coastal_states_clip[coastal_states_clip['STATE_COD'].isin(['NJ','CT','RI'])].plot(
    ax=ax,fc='#B4B3B4',ec='none',lw=1,transform=element_crs,zorder=0,rasterized=True, legend=False)

# plot LI counties
counties[counties['NAME'].isin(['Kings','Queens','Nassau','Suffolk'])].plot(
    ax=ax,fc='whitesmoke',ec='grey',lw=0,transform=element_crs,zorder=0,rasterized=True, legend=False)
hudson_river.plot(
    ax=ax, fc='#C2E8FA', ec='none', transform=element_crs, zorder=0, rasterized=True, legend=False)
water_bodies_clip.plot(
    ax=ax, fc='#C2E8FA', ec='none', transform=element_crs, zorder=0, rasterized=True, legend=False)

# try playing around with the state lines here
state_lines_clip.plot(
    ax=ax, fc='none', ec='k', linestyle=(0, (3, 4, 1, 5)), lw=1, transform=element_crs, zorder=0, rasterized=True, legend=False)


## Class activity (or own your own): Make your own figure! 

Load in some shapefiles that you'd like to display and show them on a figure by setting the appropriate extent.